**Import and installation of required libraries**

In [1]:
pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=001efc08c7a4b63a94d44019086a0901536f6675db3c4f7524f64010b23aa4ff
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [2]:
import numpy as np
import pandas as pd
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


**Download the titanic dataset**

In [3]:
from tflearn.datasets import titanic
dataset = titanic.download_dataset('titanic_dataset.csv')

Succesfully downloaded titanic_dataset.csv 82865 bytes.


In [4]:
print(dataset)

./titanic_dataset.csv


**Read the content inside titanic dataset**

In [5]:
dataframe = pd.read_csv(dataset)

In [6]:
dataframe.head()

survived  pclass                                             name     sex  \
0         1       1                    Allen, Miss. Elisabeth Walton  female   
1         1       1                   Allison, Master. Hudson Trevor    male   
2         0       1                     Allison, Miss. Helen Loraine  female   
3         0       1             Allison, Mr. Hudson Joshua Creighton    male   
4         0       1  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare  
0  29.0000      0      0   24160  211.3375  
1   0.9167      1      2  113781  151.5500  
2   2.0000      1      2  113781  151.5500  
3  30.0000      1      2  113781  151.5500  
4  25.0000      1      2  113781  151.5500

**Shape of the dataframe**

In [7]:
dataframe.shape

(1309, 9)

**Columns in dataframe**

In [8]:
dataframe.columns

Index(['survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare'],
      dtype='object')

**Load CSV file, first column represents labels**

In [9]:
from tflearn.data_utils import load_csv
data, labels = load_csv(dataset, target_column=0, categorical_labels=True, n_classes=2)

In [10]:
data

[['1',
  'Allen, Miss. Elisabeth Walton',
  'female',
  '29',
  '0',
  '0',
  '24160',
  '211.3375'],
 ['1',
  'Allison, Master. Hudson Trevor',
  'male',
  '0.9167',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1',
  'Allison, Miss. Helen Loraine',
  'female',
  '2',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1',
  'Allison, Mr. Hudson Joshua Creighton',
  'male',
  '30',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1',
  'Allison, Mrs. Hudson J C (Bessie Waldo Daniels)',
  'female',
  '25',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1', 'Anderson, Mr. Harry', 'male', '48', '0', '0', '19952', '26.5500'],
 ['1',
  'Andrews, Miss. Kornelia Theodosia',
  'female',
  '63',
  '1',
  '0',
  '13502',
  '77.9583'],
 ['1', 'Andrews, Mr. Thomas Jr', 'male', '39', '0', '0', '112050', '0.0000'],
 ['1',
  'Appleton, Mrs. Edward Dale (Charlotte Lamson)',
  'female',
  '53',
  '2',
  '0',
  '11769',
  '51.4792'],
 ['1',
  'Artagaveytia, Mr. Ramon',
  'male',
  '71',
  '0',
  '0',
  'PC 17609',
  '49

In [11]:
labels

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

**Data Preprocessing - Converting the sex feature into numerical value where male is represented as '0' and female is represented as '1'**

In [12]:
# Preprocessing function
def preprocess(data, discarded_columns):
    # Sort by descending id and delete columns
    for id in sorted(discarded_columns, reverse=True):
        [r.pop(id) for r in data]
    for i in range(len(data)):
      # Converting 'sex' field to float (id is 1 after removing labels column)
      data[i][1] = 1. if data[i][1] == 'female' else 0.
    return np.array(data, dtype=np.float32)# Ignore 'name' and 'ticket' columns (id 1 & 6 of data array)
discarded_columns =[1, 6]# Preprocess data
## function calling
data = preprocess(data, discarded_columns)

**Build the neural network classifier**

In [13]:
# Build neural network classifier(3-layer neural network)
net = tflearn.input_data(shape=[None, 6])
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


**Model Training**

In [14]:
# Define model
model = tflearn.DNN(net)
# Start training (apply gradient descent algorithm)
model.fit(data, labels, n_epoch=20, batch_size=16, show_metric=True)

Training Step: 1639  | total loss: 0.51649 | time: 0.451s
| Adam | epoch: 020 | loss: 0.51649 - acc: 0.7755 -- iter: 1296/1309
Training Step: 1640  | total loss: 0.50307 | time: 0.456s
| Adam | epoch: 020 | loss: 0.50307 - acc: 0.7854 -- iter: 1309/1309
--


**Model Prediction**

In [15]:
## loading the data
shivam = [3, 'Shivam Bhatia', 'male', 19, 0, 0, 'N/A', 5.0000]
priya = [1, 'Priya Bhatia', 'female', 17, 1, 2, 'N/A', 100.0000]
# Preprocess data
shivam, priya = preprocess([shivam, priya], discarded_columns)
# Predict surviving chances (class 1 results)
pred = model.predict([shivam, priya])
print("Shivam's Surviving Rate:", pred[0][1])
print("Priya's Surviving Rate:", pred[1][1])

Shivam's Surviving Rate: 0.08110144
Priya's Surviving Rate: 0.82581884
